In [ ]:
# Auto-detect environment (Colab vs Local)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IS_COLAB = True
    print("✅ Running on Google Colab")
except ImportError:
    IS_COLAB = False
    print("✅ Running on Local Machine")


Mounted at /content/drive


In [ ]:
# STEP 2: Install/Check PyTorch with GPU Support
try:
    import torch
    print("✅ PyTorch already installed")
except ImportError:
    print("📦 Installing PyTorch with CUDA 11.8 support...")
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
    import torch

# Check GPU Availability
print("\n🔍 GPU Detection:")
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Training will use CPU (slower)")
    if IS_COLAB:
        print("   → Go to: Runtime > Change runtime type > Select GPU")
    else:
        print("   → Ensure CUDA drivers are installed on your system")


PyTorch Version: 2.9.0+cpu
CUDA Available: False
⚠️ No GPU detected. Go to: Runtime > Change runtime type > Select GPU


In [ ]:
# STEP 2b: Check TensorFlow with GPU Support
try:
    import tensorflow as tf
    print("✅ TensorFlow already installed")
except ImportError:
    print("📦 Installing TensorFlow with GPU support...")
    !pip install tensorflow --quiet
    import tensorflow as tf

print("\n🔍 TensorFlow GPU Status:")
print("TensorFlow Version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ {len(gpus)} GPU(s) Available: {gpus}")
    for gpu in gpus:
        print(f"   GPU: {gpu}")
else:
    print("⚠️ No GPU detected by TensorFlow")


In [ ]:



# STEP 3 (corrected): Recursive loader for patient subfolders and mask pairing
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Auto-detect dataset path based on environment
if IS_COLAB:
    # Google Colab path
    LGG_ROOT = "/content/drive/MyDrive/Datasets/Brain MRI segmentation/kaggle_3m"
else:
    # Local machine path - adjust this to your actual dataset location
    # If you don't have the raw dataset locally, skip this cell and use Cell 6 instead
    LGG_ROOT = "C:/Users/KIIT/Desktop/Datasets/Brain_MRI_segmentation/kaggle_3m"
    if not os.path.exists(LGG_ROOT):
        print("⚠️ Raw dataset not found locally.")
        print("   → Skip this cell and run Cell 6 to load preprocessed data directly.")
        LGG_ROOT = None

print(f"Dataset Root: {LGG_ROOT}")

# Supported image/mask extensions (order matters: mask suffix handled separately)
IMG_EXTS = (".tif", ".tiff", ".png", ".jpg", ".jpeg")

# Containers
image_paths = []
mask_paths = []

def find_pairs_in_patient_folder(patient_folder):
    """Find (image, mask) pairs inside a patient folder.
       Masks are expected to contain '_mask' before the extension (e.g. '..._1_mask.tif').
    """
    files = sorted(os.listdir(patient_folder))
    # index files by name for fast lookup
    file_set = set(files)
    pairs = []

    for f in files:
        lf = f.lower()
        # consider only potential mask files (having '_mask' in name)
        if "_mask" in lf:
            name_no_mask, ext = os.path.splitext(f)
            # verify extension
            if ext.lower() not in IMG_EXTS:
                continue
            # derive image filename candidates by removing the last "_mask" occurrence
            # e.g. "TCGA_..._1_mask.tif" -> "TCGA_..._1.tif"
            img_base = name_no_mask.rsplit("_mask", 1)[0]
            # try same extension first
            candidate = img_base + ext
            if candidate in file_set:
                pairs.append((os.path.join(patient_folder, candidate), os.path.join(patient_folder, f)))
                continue
            # try other extensions if candidate not found
            found = False
            for e in IMG_EXTS:
                candidate_e = img_base + e
                if candidate_e in file_set:
                    pairs.append((os.path.join(patient_folder, candidate_e), os.path.join(patient_folder, f)))
                    found = True
                    break
            # if not found, skip but report later
    return pairs

# Walk one level deep in LGG_ROOT (each patient folder contains slices)
print("Scanning patient folders and pairing image/mask files...")
for entry in sorted(os.listdir(LGG_ROOT)):
    p = os.path.join(LGG_ROOT, entry)
    if os.path.isdir(p):
        pairs = find_pairs_in_patient_folder(p)
        for img_p, mask_p in pairs:
            image_paths.append(img_p)
            mask_paths.append(mask_p)

# Safety check
if len(image_paths) == 0:
    raise FileNotFoundError(f"No image/mask pairs found under {LGG_ROOT}. Please check the dataset path and folder structure.")

print(f"Total pairs found: {len(image_paths)}")

# Optional: if dataset is extremely large and limited RAM is available, a subset can be used:
# MAX_SAMPLES = 5000
# if len(image_paths) > MAX_SAMPLES:
#     image_paths = image_paths[:MAX_SAMPLES]
#     mask_paths = mask_paths[:MAX_SAMPLES]
#     print(f"Using subset of {MAX_SAMPLES} samples to conserve RAM.")

# Load, preprocess, and store in numpy arrays
IMG_SIZE = 256  # target size
images = []
masks = []

print("Loading and preprocessing images (this may take a while)...")
for img_p, mask_p in tqdm(zip(image_paths, mask_paths), total=len(image_paths)):
    # read image - keep as grayscale
    img = cv2.imread(img_p, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_p, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        # Skip corrupted or unreadable files but report
        print(f"Warning: failed to read pair: {img_p} / {mask_p}")
        continue

    # Resize
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)

    # Normalize image to [0,1]
    img = img.astype(np.float32) / 255.0

    # Binarize mask: treat any non-zero as tumour (0/1)
    mask = (mask > 0).astype(np.float32)

    # Add channel dimension
    img = np.expand_dims(img, axis=-1)   # shape (H,W,1)
    mask = np.expand_dims(mask, axis=-1) # shape (H,W,1)

    images.append(img)
    masks.append(mask)

# Convert to numpy arrays
images = np.array(images, dtype=np.float32)
masks = np.array(masks, dtype=np.float32)

print("Preprocessing finished.")
print("Images shape:", images.shape)
print("Masks shape: ", masks.shape)

# Shuffle and split: Train 70%, Val 15%, Test 15%
X_train, X_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.30, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, shuffle=True)

print("\nDataset split:")
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

# Quick sanity visualisation of 4 random samples
import random
idxs = random.sample(range(len(X_train)), min(4, len(X_train)))
fig, axs = plt.subplots(4, 2, figsize=(8, 12))

for i, idx in enumerate(idxs):
    axs[i,0].imshow(X_train[idx].squeeze(), cmap='gray')
    axs[i,0].set_title("Input")
    axs[i,1].imshow(y_train[idx].squeeze(), cmap='gray')
    axs[i,1].set_title("Mask")
    axs[i,0].axis('off'); axs[i,1].axis('off')
plt.tight_layout()
plt.show()





In [ ]:
import numpy as np
import os

# Auto-detect preprocessed data path
if IS_COLAB:
    BASE_PATH = "/content/drive/MyDrive/LGGData/processed_splits"
else:
    # Local machine - use the processed_splits folder in current directory
    BASE_PATH = "processed_splits"
    # Or use absolute path:
    # BASE_PATH = "C:/Users/KIIT/Desktop/open-source/resunet/processed_splits"

print(f"📂 Loading preprocessed data from: {BASE_PATH}")

if not os.path.exists(BASE_PATH):
    raise FileNotFoundError(f"❌ Preprocessed data not found at: {BASE_PATH}")

X_train = np.load(f"{BASE_PATH}/X_train.npy")
y_train = np.load(f"{BASE_PATH}/y_train.npy")

X_val   = np.load(f"{BASE_PATH}/X_val.npy")
y_val   = np.load(f"{BASE_PATH}/y_val.npy")

X_test  = np.load(f"{BASE_PATH}/X_test.npy")
y_test  = np.load(f"{BASE_PATH}/y_test.npy")

print("\n✅ Data loaded successfully:")
print("   Train:", X_train.shape, y_train.shape)
print("   Val  :", X_val.shape, y_val.shape)
print("   Test :", X_test.shape, y_test.shape)


Train: (2750, 256, 256, 1) (2750, 256, 256, 1)
Val  : (589, 256, 256, 1) (589, 256, 256, 1)
Test : (590, 256, 256, 1) (590, 256, 256, 1)




### Step 4: Building the ResUpNet Model (Residual U-Net + ResNet50 Encoder + Attention Skip Fusion)



In [4]:

import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as sdist
import cv2
import os
import math



def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2. * intersection + smooth) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth
    )

def combo_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)

def focal_loss(gamma=2., alpha=0.25):
    def loss_fn(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        eps = K.epsilon()
        y_pred_f = K.clip(y_pred_f, eps, 1. - eps)
        pt = tf.where(tf.equal(y_true_f, 1), y_pred_f, 1 - y_pred_f)
        w = alpha * K.pow(1. - pt, gamma)
        fl = - w * K.log(pt)
        return K.mean(fl)
    return loss_fn

def hybrid_loss(alpha=0.5, gamma=2.0):
    fl = focal_loss(gamma=gamma, alpha=0.25)
    def loss(y_true, y_pred):
        return alpha * dice_loss(y_true, y_pred) + (1.0 - alpha) * fl(y_true, y_pred)
    return loss

def iou_metric(y_true, y_pred, thresh=0.5, smooth=1e-6):
    y_pred = tf.cast(y_pred > thresh, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - inter
    return (inter + smooth) / (union + smooth)



def iou_keras(y_true, y_pred): return iou_metric(y_true, y_pred)
def precision_keras(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    predicted_positive = tf.reduce_sum(y_pred)
    return tp / (predicted_positive + K.epsilon())

def recall_keras(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    actual_positive = tf.reduce_sum(y_true)
    return tp / (actual_positive + K.epsilon())

def f1_keras(y_true, y_pred):
    p = precision_keras(y_true, y_pred)
    r = recall_keras(y_true, y_pred)
    return 2 * p * r / (p + r + K.epsilon())


def attention_gate(x, g, inter_channels):
    """
    Simple attention gate as in Attention U-Net.
    x: skip connection feature map
    g: gating signal (decoder)
    """
    theta_x = layers.Conv2D(inter_channels, 1, strides=1, padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, 1, strides=1, padding='same')(g)
    add = layers.Add()([theta_x, phi_g])
    relu = layers.Activation('relu')(add)
    psi = layers.Conv2D(1, 1, strides=1, padding='same')(relu)
    sig = layers.Activation('sigmoid')(psi)


    out = layers.Multiply()([x, sig])
    return out



def residual_conv_block(x, filters, kernel_size=3):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    # Project shortcut if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


def build_resupnet(input_shape=(256,256,1), pretrained=True, train_encoder=True):
    """
    ResUpNet:
     - input_shape: grayscale (H,W,1)
     - pretrained: use ImageNet weights for ResNet50 encoder
     - train_encoder: whether encoder layers are trainable
    """
    # Input and convert to 3-channel required by ResNet50
    inp = layers.Input(shape=input_shape, name='input_image')
    # repeat channels to make 3 channels
    x = layers.Concatenate()([inp, inp, inp])  # shape -> (H,W,3)

    # Pretrained ResNet50 encoder
    base = ResNet50(include_top=False, weights='imagenet' if pretrained else None, input_tensor=x)
    base.trainable = train_encoder

    skips = [
        base.get_layer('conv1_relu').output,         # 128x128
        base.get_layer('conv2_block3_out').output,   # 64x64
        base.get_layer('conv3_block4_out').output,   # 32x32
        base.get_layer('conv4_block6_out').output    # 16x16
    ]
    bottleneck = base.get_layer('conv5_block3_out').output  # 8x8

    # Decoder: progressive upsampling, attention on skip, residual conv blocks
    d = bottleneck
    filters = [512, 256, 128, 64]  # matching decoder filters

    for i, f in enumerate(filters):
        # upsample
        d = layers.UpSampling2D(size=(2,2), interpolation='bilinear')(d)
        # attention gate on skip connection
        skip = skips[-(i+1)]
        att = attention_gate(skip, d, inter_channels=f//4)
        # concatenate
        d = layers.Concatenate()([d, att])
        # residual conv block
        d = residual_conv_block(d, f)

    # Final upsample to original resolution (from 128 -> 256 if needed)
    d = layers.UpSampling2D(size=(2,2), interpolation='bilinear')(d)
    d = residual_conv_block(d, 32)

    # Output
    out = layers.Conv2D(1, (1,1), padding='same', activation='sigmoid', name='mask')(d)

    model = models.Model(inputs=inp, outputs=out, name='ResUpNet')
    return model






In [ ]:
model = build_resupnet()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=combo_loss,
    metrics=[
        'accuracy',
        dice_coef,
        tf.keras.metrics.MeanIoU(num_classes=2)
    ]
)

model.summary()


In [ ]:
#2nd trial

In [ ]:
import numpy as np
import scipy.spatial.distance as sdist
from skimage import measure



def dice_np(y_true, y_pred, smooth=1e-6):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    inter = np.sum(y_true_f * y_pred_f)
    return (2. * inter + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def iou_np(y_true, y_pred, smooth=1e-6):
    inter = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - inter
    return (inter + smooth) / (union + smooth)

def precision_np(y_true, y_pred, smooth=1e-6):
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    return tp / (tp + fp + smooth)

def recall_np(y_true, y_pred, smooth=1e-6):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    return tp / (tp + fn + smooth)

def f1_np(y_true, y_pred, smooth=1e-6):
    p = precision_np(y_true, y_pred)
    r = recall_np(y_true, y_pred)
    return (2 * p * r) / (p + r + smooth)

def specificity_np(y_true, y_pred, smooth=1e-6):
    tn = np.sum((1 - y_true) * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    return tn / (tn + fp + smooth)

def hd95_np(y_true, y_pred):
    y_true_pts = np.argwhere(y_true > 0)
    y_pred_pts = np.argwhere(y_pred > 0)

    if len(y_true_pts) == 0 or len(y_pred_pts) == 0:
        return 0.0  # safe fallback

    d1 = sdist.cdist(y_true_pts, y_pred_pts)
    d2 = sdist.cdist(y_pred_pts, y_true_pts)
    return max(np.percentile(d1.min(axis=1), 95),
               np.percentile(d2.min(axis=1), 95))

def asd_np(y_true, y_pred):
    """
    Average Surface Distance (ASD)
    Computes mean bidirectional surface distance
    """

    # Ensure binary
    y_true = y_true.squeeze()
    y_pred = y_pred.squeeze()

    # Extract contours (surface pixels)
    true_contours = measure.find_contours(y_true, 0.5)
    pred_contours = measure.find_contours(y_pred, 0.5)

    if len(true_contours) == 0 or len(pred_contours) == 0:
        return 0.0  # safe fallback (no tumor)

    # Stack contour points
    true_pts = np.vstack(true_contours)
    pred_pts = np.vstack(pred_contours)

    # Pairwise distances
    d_true_to_pred = sdist.cdist(true_pts, pred_pts)
    d_pred_to_true = sdist.cdist(pred_pts, true_pts)

    # Mean of minimum distances
    asd = (np.mean(d_true_to_pred.min(axis=1)) +
           np.mean(d_pred_to_true.min(axis=1))) / 2.0

    return asd

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.spatial.distance as sdist

class EpochEvaluationCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, threshold=0.5, max_samples=None):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.threshold = threshold
        self.max_samples = max_samples


    def on_epoch_end(self, epoch, logs=None):
        all_dice, all_iou, all_prec, all_rec, all_f1 = [], [], [], [], []
        all_hd95, all_asd = [], []

        idxs = range(len(self.X_val))
        if self.max_samples:
            idxs = list(idxs)[:self.max_samples]

        for i in idxs:
            x = self.X_val[i:i+1]
            y_true = self.y_val[i].squeeze()

            y_prob = self.model.predict(x, verbose=0)[0, ..., 0]
            y_pred = (y_prob > self.threshold).astype(np.float32)

            d = dice_np(y_true, y_pred)
            j = iou_np(y_true, y_pred)
            p = precision_np(y_true, y_pred)
            r = recall_np(y_true, y_pred)
            f1 = f1_np(y_true, y_pred)

            h = hd95_np(y_true, y_pred)
            a = asd_np(y_true, y_pred)

            all_dice.append(d)
            all_iou.append(j)
            all_prec.append(p)
            all_rec.append(r)
            all_f1.append(f1)
            all_hd95.append(h)
            all_asd.append(a)

        print(f"\n📊 Epoch {epoch+1} — Validation Metrics:")
        print(f"Dice:      {np.nanmean(all_dice):.4f}")
        print(f"IoU:       {np.nanmean(all_iou):.4f}")
        print(f"Precision: {np.nanmean(all_prec):.4f}")
        print(f"Recall:    {np.nanmean(all_rec):.4f}")
        print(f"F1:        {np.nanmean(all_f1):.4f}")
        print(f"HD95(px):  {np.nanmean(all_hd95):.2f}")
        print(f"ASD(px):   {np.nanmean(all_asd):.2f}")


epoch_eval_cb = EpochEvaluationCallback(
    X_val, y_val,
    threshold=0.5,
    max_samples=50
)

In [ ]:
epoch_eval_cb = EpochEvaluationCallback(
    X_val, y_val,
    threshold=0.5,# or equal to 0.45(optimal-callback rerun it in case of bad outputs)
    max_samples=50
)


In [ ]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping
)

import scipy.spatial.distance as sdist
import matplotlib.pyplot as plt

callbacks = [
    ModelCheckpoint(
        "best_resupnet.keras",
        monitor="val_dice_coef",
        save_best_only=True,
        mode="max",
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor="val_dice_coef",
        factor=0.50,
        patience=3,
        min_lr=1e-6,
        mode="max",
        verbose=1
    ),EarlyStopping(
    monitor="val_dice_coef",
    mode="max",
    patience=8,
    restore_best_weights=True,
    verbose=1
)
,
    epoch_eval_cb
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=8,
    shuffle=True,
    callbacks=callbacks
)


In [ ]:
### NORMAL PLOTS

In [ ]:
history_dict = history.history

train_loss = history_dict['loss']
val_loss   = history_dict['val_loss']

train_dice = history_dict['dice_coef']
val_dice   = history_dict['val_dice_coef']

epochs = range(1, len(train_loss) + 1)

In [ ]:
lrs = []
optimizer = model.optimizer

for i in range(len(epochs)):
    lrs.append(tf.keras.backend.get_value(optimizer.learning_rate))


In [ ]:
dice_gap = np.array(train_dice) - np.array(val_dice)
loss_gap = np.array(val_loss) - np.array(train_loss)


In [ ]:
best_val_dice = []
current_best = 0

for d in val_dice:
    current_best = max(current_best, d)
    best_val_dice.append(current_best)


In [ ]:
plt.figure(figsize=(20, 12))

# ---- 1. Training vs Validation Loss ----
plt.subplot(2, 3, 1)
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)

# ---- 2. Training vs Validation Dice ----
plt.subplot(2, 3, 2)
plt.plot(epochs, train_dice, 'bo-', label='Training Dice Coef')
plt.plot(epochs, val_dice, 'ro-', label='Validation Dice Coef')
plt.xlabel('Epoch')
plt.ylabel('Dice Coefficient')
plt.title('Training vs Validation Dice Coefficient')
plt.legend()
plt.grid(True)

# ---- 3. Learning Rate Schedule ----
plt.subplot(2, 3, 3)
plt.plot(epochs, lrs, 'mo-', linewidth=2)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule (ReduceLROnPlateau)')
plt.grid(True)

# ---- 4. Dice Generalization Gap ----
plt.subplot(2, 3, 4)
plt.plot(epochs, dice_gap, color='orange', marker='o')
plt.fill_between(epochs, dice_gap, alpha=0.3, color='orange')
plt.axhline(0, linestyle='--', color='black')
plt.xlabel('Epoch')
plt.ylabel('Dice Gap (Train - Val)')
plt.title('Generalization Gap (Dice Coefficient)')
plt.grid(True)

# ---- 5. Loss Generalization Gap ----
plt.subplot(2, 3, 5)
plt.plot(epochs, loss_gap, color='salmon', marker='o')
plt.fill_between(epochs, loss_gap, alpha=0.3, color='salmon')
plt.axhline(0, linestyle='--', color='black')
plt.xlabel('Epoch')
plt.ylabel('Loss Gap (Val - Train)')
plt.title('Generalization Gap (Loss)')
plt.grid(True)

# ---- 6. Best Model Progression ----
plt.subplot(2, 3, 6)
plt.plot(epochs, best_val_dice, 'g*-', linewidth=2)
for i, v in enumerate(best_val_dice):
    if i % 5 == 0 or i == len(best_val_dice) - 1:
        plt.text(i + 1, v, f"{v:.4f}", fontsize=9)
plt.xlabel('Epoch')
plt.ylabel('Validation Dice Coefficient')
plt.title('Best Model Progression')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
###COMPARISIONPLOTS###

In [ ]:
import matplotlib.pyplot as plt

def plot_training(history):
    epochs = range(1, len(history.history['loss']) + 1)

# training vs epoch plots
    plt.figure(figsize=(6,5))
    plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('accuracy_curve.png', dpi=300)
    plt.show()

    # loss plots
    plt.figure(figsize=(6,5))
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('loss_curve.png', dpi=300)
    plt.show()


    if 'mean_io_u' in history.history:
        plt.figure(figsize=(6,5))
        plt.plot(epochs, history.history['mean_io_u'], label='Training IoU')
        plt.plot(epochs, history.history['val_mean_io_u'], label='Validation IoU')
        plt.xlabel('Epochs')
        plt.ylabel('Mean IoU')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('iou_curve.png', dpi=300)
        plt.show()


In [ ]:
#### BLAND ANALYSIS PLOTS

In [ ]:
def compute_volumes(y_true, y_pred):
    """
    y_true, y_pred: binary masks (H, W) or (H, W, D)
    returns volume (number of positive pixels/voxels)
    """
    return np.sum(y_true), np.sum(y_pred)


In [ ]:
gt_volumes = []
pred_volumes = []

threshold = 0.5
max_samples = 50

for i in range(max_samples):
    x = X_val[i:i+1]
    y_true = y_val[i].squeeze()

    y_prob = model.predict(x, verbose=0)[0, ..., 0]
    y_pred = (y_prob > threshold).astype(np.float32)

    gt_v, pred_v = compute_volumes(y_true, y_pred)

    gt_volumes.append(gt_v)
    pred_volumes.append(pred_v)

gt_volumes = np.array(gt_volumes)
pred_volumes = np.array(pred_volumes)


In [ ]:
means = (gt_volumes + pred_volumes) / 2
diffs = pred_volumes - gt_volumes

mean_diff = np.mean(diffs)
std_diff = np.std(diffs)

loa_upper = mean_diff + 1.96 * std_diff
loa_lower = mean_diff - 1.96 * std_diff


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,5))

plt.scatter(means, diffs, alpha=0.7)
plt.axhline(mean_diff, linestyle='--', linewidth=2, label=f'Mean diff = {mean_diff:.2f}')
plt.axhline(loa_upper, linestyle='--', linewidth=2, label=f'+1.96 SD = {loa_upper:.2f}')
plt.axhline(loa_lower, linestyle='--', linewidth=2, label=f'-1.96 SD = {loa_lower:.2f}')

plt.xlabel('Mean of GT and Predicted Volume')
plt.ylabel('Difference (Pred − GT)')
plt.title('Bland–Altman Analysis')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('bland_altman_volume.png', dpi=300)
plt.show()
### for ground truth image this plot is needed

### Additional Visualizations - Test Set Analysis

In [ ]:
# 1. Distribution of Metrics Across Test Set
print("Computing metrics for all test samples...")
test_dice, test_iou, test_prec, test_rec = [], [], [], []
test_f1, test_hd95, test_asd = [], [], []

threshold = 0.5

for i in range(len(X_test)):
    x = X_test[i:i+1]
    y_true = y_test[i].squeeze()
    
    y_prob = model.predict(x, verbose=0)[0, ..., 0]
    y_pred = (y_prob > threshold).astype(np.float32)
    
    test_dice.append(dice_np(y_true, y_pred))
    test_iou.append(iou_np(y_true, y_pred))
    test_prec.append(precision_np(y_true, y_pred))
    test_rec.append(recall_np(y_true, y_pred))
    test_f1.append(f1_np(y_true, y_pred))
    test_hd95.append(hd95_np(y_true, y_pred))
    test_asd.append(asd_np(y_true, y_pred))

print("✅ Metrics computed for all test samples")

In [ ]:
# 2. Box Plots for All Metrics
plt.figure(figsize=(16, 10))

metrics_data = {
    'Dice': test_dice,
    'IoU': test_iou,
    'Precision': test_prec,
    'Recall': test_rec,
    'F1': test_f1
}

# Subplot 1: Main segmentation metrics
plt.subplot(2, 2, 1)
plt.boxplot(metrics_data.values(), labels=metrics_data.keys())
plt.ylabel('Score')
plt.title('Distribution of Segmentation Metrics on Test Set')
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.05])

# Add mean values
for i, (name, values) in enumerate(metrics_data.items(), 1):
    mean_val = np.mean(values)
    plt.text(i, mean_val, f'{mean_val:.3f}', ha='center', va='bottom', fontweight='bold')

# Subplot 2: Distance metrics
plt.subplot(2, 2, 2)
plt.boxplot([test_hd95, test_asd], labels=['HD95 (px)', 'ASD (px)'])
plt.ylabel('Distance (pixels)')
plt.title('Distribution of Distance Metrics on Test Set')
plt.grid(True, alpha=0.3)

# Subplot 3: Dice Score Distribution (Histogram)
plt.subplot(2, 2, 3)
plt.hist(test_dice, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(np.mean(test_dice), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(test_dice):.4f}')
plt.axvline(np.median(test_dice), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(test_dice):.4f}')
plt.xlabel('Dice Coefficient')
plt.ylabel('Frequency')
plt.title('Dice Score Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: IoU Score Distribution (Histogram)
plt.subplot(2, 2, 4)
plt.hist(test_iou, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
plt.axvline(np.mean(test_iou), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(test_iou):.4f}')
plt.axvline(np.median(test_iou), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(test_iou):.4f}')
plt.xlabel('IoU Score')
plt.ylabel('Frequency')
plt.title('IoU Score Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('test_metrics_distribution.png', dpi=300)
plt.show()

In [ ]:
# 3. Correlation Heatmap Between Metrics
import pandas as pd
import seaborn as sns

metrics_df = {
    'Dice': test_dice,
    'IoU': test_iou,
    'Precision': test_prec,
    'Recall': test_rec,
    'F1': test_f1,
    'HD95': test_hd95,
    'ASD': test_asd
}

# Create correlation matrix
df = pd.DataFrame(metrics_df)
corr_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Evaluation Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('metrics_correlation_heatmap.png', dpi=300)
plt.show()

In [ ]:
# 4. Scatter Plot: Predicted vs Ground Truth Volumes
gt_test_volumes = []
pred_test_volumes = []

for i in range(len(X_test)):
    x = X_test[i:i+1]
    y_true = y_test[i].squeeze()
    
    y_prob = model.predict(x, verbose=0)[0, ..., 0]
    y_pred = (y_prob > threshold).astype(np.float32)
    
    gt_v, pred_v = compute_volumes(y_true, y_pred)
    gt_test_volumes.append(gt_v)
    pred_test_volumes.append(pred_v)

gt_test_volumes = np.array(gt_test_volumes)
pred_test_volumes = np.array(pred_test_volumes)

plt.figure(figsize=(8, 8))
plt.scatter(gt_test_volumes, pred_test_volumes, alpha=0.6, s=50, edgecolors='black')

# Perfect prediction line
max_vol = max(gt_test_volumes.max(), pred_test_volumes.max())
plt.plot([0, max_vol], [0, max_vol], 'r--', linewidth=2, label='Perfect Prediction')

# Add correlation coefficient
corr, p_value = pearsonr(gt_test_volumes, pred_test_volumes)
plt.text(0.05, 0.95, f'Pearson r = {corr:.4f}\np-value = {p_value:.4e}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.xlabel('Ground Truth Volume (pixels)', fontsize=12)
plt.ylabel('Predicted Volume (pixels)', fontsize=12)
plt.title('Predicted vs Ground Truth Tumor Volume', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('volume_scatter_plot.png', dpi=300)
plt.show()

In [ ]:
# 5. Best, Worst, and Median Predictions Visualization
# Find best, worst, and median cases based on Dice score
dice_with_idx = [(d, i) for i, d in enumerate(test_dice)]
dice_with_idx.sort(key=lambda x: x[0])

worst_idx = dice_with_idx[0][1]
median_idx = dice_with_idx[len(dice_with_idx)//2][1]
best_idx = dice_with_idx[-1][1]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))

cases = [
    ('Worst', worst_idx, test_dice[worst_idx]),
    ('Median', median_idx, test_dice[median_idx]),
    ('Best', best_idx, test_dice[best_idx])
]

for row, (label, idx, dice_score) in enumerate(cases):
    x = X_test[idx:idx+1]
    y_true = y_test[idx].squeeze()
    
    y_prob = model.predict(x, verbose=0)[0, ..., 0]
    y_pred = (y_prob > threshold).astype(np.float32)
    
    # Original image
    axes[row, 0].imshow(X_test[idx].squeeze(), cmap='gray')
    axes[row, 0].set_title(f'{label} Case - Input\nDice: {dice_score:.4f}')
    axes[row, 0].axis('off')
    
    # Ground truth
    axes[row, 1].imshow(y_true, cmap='gray')
    axes[row, 1].set_title('Ground Truth')
    axes[row, 1].axis('off')
    
    # Prediction
    axes[row, 2].imshow(y_pred, cmap='gray')
    axes[row, 2].set_title('Prediction')
    axes[row, 2].axis('off')
    
    # Overlay
    overlay = X_test[idx].squeeze()
    axes[row, 3].imshow(overlay, cmap='gray')
    axes[row, 3].contour(y_true, colors='green', linewidths=2, alpha=0.7, label='GT')
    axes[row, 3].contour(y_pred, colors='red', linewidths=2, alpha=0.7, label='Pred')
    axes[row, 3].set_title('Overlay (Green=GT, Red=Pred)')
    axes[row, 3].axis('off')

plt.tight_layout()
plt.savefig('best_worst_median_predictions.png', dpi=300)
plt.show()

In [ ]:
# 6. ROC Curve and Precision-Recall Curve
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

# Collect all predictions and ground truths
all_y_true = []
all_y_pred_prob = []

print("Generating probability predictions for ROC/PR curves...")
for i in range(len(X_test)):
    x = X_test[i:i+1]
    y_true = y_test[i].squeeze()
    
    y_prob = model.predict(x, verbose=0)[0, ..., 0]
    
    all_y_true.extend(y_true.flatten())
    all_y_pred_prob.extend(y_prob.flatten())

all_y_true = np.array(all_y_true)
all_y_pred_prob = np.array(all_y_pred_prob)

# ROC Curve
fpr, tpr, thresholds_roc = roc_curve(all_y_true, all_y_pred_prob)
roc_auc = auc(fpr, tpr)

# Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(all_y_true, all_y_pred_prob)
avg_precision = average_precision_score(all_y_true, all_y_pred_prob)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# ROC Curve
ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate', fontsize=12)
ax1.set_ylabel('True Positive Rate', fontsize=12)
ax1.set_title('ROC Curve', fontsize=14, fontweight='bold')
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AP = {avg_precision:.4f})')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall', fontsize=12)
ax2.set_ylabel('Precision', fontsize=12)
ax2.set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
ax2.legend(loc="lower left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('roc_pr_curves.png', dpi=300)
plt.show()

print(f"\n📊 ROC AUC: {roc_auc:.4f}")
print(f"📊 Average Precision: {avg_precision:.4f}")

In [ ]:
# 7. Confusion Matrix (Pixel-wise)
from sklearn.metrics import confusion_matrix

# Binarize predictions
all_y_pred_binary = (all_y_pred_prob > threshold).astype(int)

cm = confusion_matrix(all_y_true, all_y_pred_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True, 
            xticklabels=['Background', 'Tumor'], 
            yticklabels=['Background', 'Tumor'],
            cbar_kws={'label': 'Count'})
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.title('Confusion Matrix (Pixel-wise)', fontsize=14, fontweight='bold')

# Add percentages
total = cm.sum()
for i in range(2):
    for j in range(2):
        percentage = (cm[i, j] / total) * 100
        plt.text(j + 0.5, i + 0.7, f'({percentage:.2f}%)', 
                ha='center', va='center', fontsize=10, color='red')

plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

# Calculate specificity
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

print(f"\n📊 Pixel-wise Metrics:")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

In [ ]:
# 8. Error Analysis - Cases with Low Dice Scores
low_dice_threshold = 0.70
low_dice_cases = [(d, i) for i, d in enumerate(test_dice) if d < low_dice_threshold]

if len(low_dice_cases) > 0:
    print(f"Found {len(low_dice_cases)} cases with Dice < {low_dice_threshold}")
    
    # Visualize up to 9 low-performing cases
    n_display = min(9, len(low_dice_cases))
    low_dice_cases_sorted = sorted(low_dice_cases, key=lambda x: x[0])[:n_display]
    
    rows = int(np.ceil(n_display / 3))
    fig, axes = plt.subplots(rows, 3, figsize=(15, 5 * rows))
    axes = axes.flatten() if n_display > 1 else [axes]
    
    for plot_idx, (dice_score, idx) in enumerate(low_dice_cases_sorted):
        x = X_test[idx:idx+1]
        y_true = y_test[idx].squeeze()
        
        y_prob = model.predict(x, verbose=0)[0, ..., 0]
        y_pred = (y_prob > threshold).astype(np.float32)
        
        # Overlay visualization
        overlay = X_test[idx].squeeze()
        axes[plot_idx].imshow(overlay, cmap='gray')
        axes[plot_idx].contour(y_true, colors='green', linewidths=2, alpha=0.7)
        axes[plot_idx].contour(y_pred, colors='red', linewidths=2, alpha=0.7)
        axes[plot_idx].set_title(f'Sample {idx}\nDice: {dice_score:.4f}\nIoU: {test_iou[idx]:.4f}')
        axes[plot_idx].axis('off')
    
    # Hide unused subplots
    for i in range(n_display, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'Error Analysis - Cases with Dice < {low_dice_threshold}\n(Green=GT, Red=Pred)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('error_analysis_low_dice.png', dpi=300)
    plt.show()
else:
    print(f"✅ All test cases have Dice >= {low_dice_threshold}!")

In [ ]:
# 9. Comprehensive Summary Statistics Table
import pandas as pd

print("\n" + "="*60)
print("📊 COMPREHENSIVE TEST SET EVALUATION SUMMARY")
print("="*60)

summary_stats = {
    'Metric': ['Dice Coefficient', 'IoU', 'Precision', 'Recall', 'F1 Score', 
               'HD95 (pixels)', 'ASD (pixels)', 'Specificity'],
    'Mean': [
        np.mean(test_dice), 
        np.mean(test_iou), 
        np.mean(test_prec), 
        np.mean(test_rec), 
        np.mean(test_f1),
        np.mean(test_hd95), 
        np.mean(test_asd),
        specificity
    ],
    'Std': [
        np.std(test_dice), 
        np.std(test_iou), 
        np.std(test_prec), 
        np.std(test_rec), 
        np.std(test_f1),
        np.std(test_hd95), 
        np.std(test_asd),
        0  # Single value
    ],
    'Median': [
        np.median(test_dice), 
        np.median(test_iou), 
        np.median(test_prec), 
        np.median(test_rec), 
        np.median(test_f1),
        np.median(test_hd95), 
        np.median(test_asd),
        specificity
    ],
    'Min': [
        np.min(test_dice), 
        np.min(test_iou), 
        np.min(test_prec), 
        np.min(test_rec), 
        np.min(test_f1),
        np.min(test_hd95), 
        np.min(test_asd),
        specificity
    ],
    'Max': [
        np.max(test_dice), 
        np.max(test_iou), 
        np.max(test_prec), 
        np.max(test_rec), 
        np.max(test_f1),
        np.max(test_hd95), 
        np.max(test_asd),
        specificity
    ]
}

summary_df = pd.DataFrame(summary_stats)
print(summary_df.to_string(index=False))
print("="*60)

# Save to CSV
summary_df.to_csv('test_evaluation_summary.csv', index=False)
print("\n✅ Summary saved to 'test_evaluation_summary.csv'")

In [ ]:
# 10. Violin Plot - Detailed Distribution Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Segmentation metrics violin plot
ax1 = axes[0]
parts1 = ax1.violinplot([test_dice, test_iou, test_prec, test_rec, test_f1], 
                         positions=range(1, 6), showmeans=True, showmedians=True)

for pc in parts1['bodies']:
    pc.set_facecolor('lightblue')
    pc.set_alpha(0.7)

ax1.set_xticks(range(1, 6))
ax1.set_xticklabels(['Dice', 'IoU', 'Precision', 'Recall', 'F1'])
ax1.set_ylabel('Score', fontsize=12)
ax1.set_title('Violin Plot - Segmentation Metrics', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim([0, 1.05])

# Distance metrics violin plot
ax2 = axes[1]
parts2 = ax2.violinplot([test_hd95, test_asd], positions=[1, 2], 
                         showmeans=True, showmedians=True)

for pc in parts2['bodies']:
    pc.set_facecolor('lightcoral')
    pc.set_alpha(0.7)

ax2.set_xticks([1, 2])
ax2.set_xticklabels(['HD95', 'ASD'])
ax2.set_ylabel('Distance (pixels)', fontsize=12)
ax2.set_title('Violin Plot - Distance Metrics', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('violin_plots_metrics.png', dpi=300)
plt.show()